<a href="https://colab.research.google.com/github/alyson-singleton/IOH_and_dengue/blob/main/00_IOH_and_dengue_GEE_downloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from IPython.display import display
display({'application/vnd.jupyter.widget-state+json': {}}, raw=True)

In [1]:
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gbsc-gcp-lab-emordeca')

In [18]:
##############################
# 1. Load Spatial Data
##############################

# Load point locations (DIRESA coordinates)
# (located in github "data/spatial_data" folder, upload to your GEE account)
mdd_points = ee.FeatureCollection('users/asinglet/diresa_esalud_coordinates_key')

# Buffer each point by 5 km (with max error 100m)
mdd_spatial_units = mdd_points.map(lambda f: f.buffer(5000, 100))

# Load Madre de Dios department boundary
# (located in github "data/spatial_data" folder, upload to your GEE account)
mdd_department = ee.FeatureCollection("users/asinglet/peru_departments") \
    .filter(ee.Filter.eq('NOMBDEP', 'MADRE DE DIOS'))

# Load PE-30C highway within Madre de Dios
# (located in github "data/spatial_data" folder, upload to your GEE account)
mdd_IOH = ee.FeatureCollection('users/asinglet/peru_roads_important') \
    .filter(ee.Filter.eq('ref', 'PE-30C')) \
    .filterBounds(mdd_department)

In [19]:
Map = geemap.Map(center=[-12.5, -70], zoom=8)
Map.addLayer(mdd_spatial_units, {'color': 'red'}, 'mdd_spatial_units')
Map.addLayer(mdd_points, {}, 'mdd_points')
Map.addLayer(mdd_IOH, {'color': 'blue'}, 'Interoceanic Highway')
Map

Map(center=[-12.5, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [11]:
##############################
# 2. Population Data (WorldPop 100m)
##############################

# Load WorldPop data from 2000–2019
pop_data = ee.ImageCollection("WorldPop/GP/100m/pop") \
    .filterDate("2000-01-01", "2019-12-31") \
    .toBands()

# Load 2020 separately
pop_data_2020 = ee.ImageCollection("WorldPop/GP/100m/pop") \
    .filterDate("2020-01-01", "2020-12-31") \
    .toBands()

# Select Peru-specific bands (indices may vary; adjust if needed)
peru_pop = pop_data.select(ee.List.sequence(3475, 3494))
peru_pop_2020 = pop_data_2020.select([174])  # Confirm correct index

# Combine all years into one image
pop_all_years = ee.Image(peru_pop).addBands(peru_pop_2020)

# Reduce to spatial units (sum of population)
pop_stats = pop_all_years.reduceRegions(
    collection=mdd_spatial_units,
    reducer=ee.Reducer.sum(),
    scale=100
)

# Export
ee.batch.Export.table.toDrive(
    collection=pop_stats,
    description='mdd_population_yearly_sum',
    folder='GEEexports',
    fileFormat='CSV'
).start()

In [12]:
##############################
# 3. Precipitation Data (ERA5-Land Monthly)
##############################

precip = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
    .select('total_precipitation_sum') \
    .toBands() \
    .select(ee.List.sequence(599, 874))

# Clip to department
precip = precip.clip(mdd_department)

# Reduce to spatial units (monthly sum of precip)
precip_stats = precip.reduceRegions(
    collection=mdd_spatial_units,
    reducer=ee.Reducer.sum(),
    scale=11132
)

# Export
ee.batch.Export.table.toDrive(
    collection=precip_stats,
    description='mdd_precipitation_monthly_sum',
    folder='GEEexports',
    fileFormat='CSV'
).start()

In [13]:
##############################
# 4. Temperature Data (ERA5-Land Monthly)
##############################

temp = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
    .select('temperature_2m') \
    .toBands() \
    .select(ee.List.sequence(599, 874))

# Clip to department
temp = temp.clip(mdd_department)

# Reduce to spatial units (monthly mean temperature)
temp_stats = temp.reduceRegions(
    collection=mdd_spatial_units,
    reducer=ee.Reducer.mean(),
    scale=11132
)

# Export
ee.batch.Export.table.toDrive(
    collection=temp_stats,
    description='mdd_temperature_monthly_mean',
    folder='GEEexports',
    fileFormat='CSV'
).start()

In [16]:
##########################################
# 5. Land Cover Data (MapBiomas RAISG v5)
##########################################

mapbiomas = ee.Image('projects/mapbiomas-raisg/public/collection5/mapbiomas_raisg_panamazonia_collection5_integration_v1').selfMask()
pixel_area = ee.Image.pixelArea().divide(1e6)  # km^2
attribute = 'key'
territories = mdd_spatial_units

# Years and classes of interest
years = list(map(str, range(2000, 2023)))
class_ids = [3, 15, 18, 21, 24]  # Forest, pasture, ag, mosaic, urban
scale = 30

# Function to convert grouped dictionary to feature collection
def convert2table(obj):
    obj = ee.Dictionary(obj)
    territory_val = obj.get('territory')
    class_area_list = ee.List(obj.get('groups'))

    def format_entry(class_area):
        class_area = ee.Dictionary(class_area)
        return ee.Feature(None, {
            attribute: territory_val,
            'class': class_area.get('class'),
            'area': class_area.get('sum')
        })

    return ee.FeatureCollection(class_area_list.map(format_entry))

# Function to calculate area by land cover class within a single territory
def calculate_area(image, territory_img, geom):
    reducer = ee.Reducer.sum().group(1, 'class').group(1, 'territory')
    data = pixel_area.addBands(territory_img).addBands(image) \
        .reduceRegion(
            reducer=reducer,
            geometry=geom,
            scale=scale,
            maxPixels=1e12
        )
    grouped = ee.List(data.get('groups'))
    return ee.FeatureCollection(grouped.map(convert2table)).flatten()

# Process each year
area_collections = []

for year in years:
    lc = mapbiomas.select(f'classification_{year}').remap(class_ids, class_ids, 0)

    def process_feature(feat):
        territory_img = ee.Image().int64().paint(featureCollection=ee.FeatureCollection(feat), color=attribute)
        return calculate_area(lc, territory_img, feat.geometry())

    yearly_stats = territories.map(process_feature).flatten()
    yearly_stats = yearly_stats.map(lambda f: f.set('year', year))
    area_collections.append(yearly_stats)

# Combine all years
final_areas = ee.FeatureCollection(area_collections).flatten()

# Export to Drive
task = ee.batch.Export.table.toDrive(
    collection=final_areas,
    description='mdd_mapbiomas_yearly_sum',
    folder='GEEexports',
    fileNamePrefix='mdd_mapbiomas_yearly_sum',
    fileFormat='CSV'
)
task.start()

In [21]:
##########################################
# 6. Highway buffer zones
##########################################

def buffer_and_check(distance_m):
    buffer = mdd_IOH.map(lambda f: f.buffer(distance_m, 100))
    layer_name = f'{distance_m // 1000} km Buffer'
    Map.addLayer(buffer, {'color': 'gray'}, layer_name)

    def set_flag(feat):
        inside = feat.geometry().intersects(buffer.geometry())
        return feat.set(f'isInside_{distance_m//1000}km', inside)

    flagged = mdd_points.map(set_flag)

    # Export table
    task = ee.batch.Export.table.toDrive(
        collection=flagged,
        description=f'mdd_IOH_buffered_{distance_m//1000}km',
        folder='GEEexports',
        fileNamePrefix=f'mdd_IOH_buffered_{distance_m//1000}km',
        fileFormat='CSV'
    )
    task.start()
    print(f'Export started for {distance_m//1000} km buffer')

########################################
# Loop over distances and run analysis
########################################

for km in [1, 2, 3, 4, 5, 10, 15, 20, 30, 40]:
    buffer_and_check(km * 1000)

# Show the map
Map

Export started for 1 km buffer
Export started for 2 km buffer
Export started for 3 km buffer
Export started for 4 km buffer
Export started for 5 km buffer
Export started for 10 km buffer
Export started for 15 km buffer
Export started for 20 km buffer
Export started for 30 km buffer
Export started for 40 km buffer


Map(bottom=35362.0, center=[-12.5, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…